# Capstone Project | Coursera

## Introduction

This notebook will be mainly used for the capstone project.

## Setup

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Clustering

### Preparation

In [23]:
import socket  
import time  
timeout = 200    
socket.setdefaulttimeout(timeout)  
sleep_download_time = 100  
time.sleep(sleep_download_time) 

In [5]:
import requests
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen

# access wiki
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
headers = {'User-agent' : 'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:22.0) Gecko/20100101 Firefox/22.0'}

request = urllib.request.Request(url, headers=headers) 
request = urlopen(request)
html = request.read().decode("utf-8")
request.close()

soup = BeautifulSoup(html, features = "lxml")

In [6]:
# dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
df = pd.DataFrame(columns=column_names)
df

,PostalCode,Borough,Neighborhood


In [7]:
tab = soup.find('table')
tab_items = tab.find_all('td')

postcodes = []
boroughs = []
neighbors = []

for index, item in enumerate(tab_items):
    if index%3==0:
        postcodes.append(item.text.strip())
    elif index%3==1:
        boroughs.append(item.text.strip())
    else:
        neighbors.append(item.text.strip())
    
print("Length: postcodes-{}, boroughs-{}, neighbors-{}".format(len(postcodes), len(boroughs), len(neighbors)))

Length: postcodes-287, boroughs-287, neighbors-287


In [8]:
df.PostalCode = postcodes
df.Borough = boroughs
df.Neighborhood = neighbors
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
# ignore cells with a borough that is Not assigned
df_filtered = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_filtered.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [10]:
# if more than one neighborhood exists in one postal code area, 
#   these two rows will be combined into one row with the neighborhoods separated with a comma
df_combined = df_filtered.copy()

postal_hash = {}
for index, row in df_filtered.iterrows():
    if row['PostalCode'] in postal_hash:
        df_combined.drop(index, inplace=True)
        df_combined.loc[postal_hash[row['PostalCode']], 'Neighborhood'] += ", " + row['Neighborhood']
    else:
        postal_hash[row['PostalCode']] = index
        
df_combined.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [11]:
# if a cell has a borough but a Not assigned neighborhood, 
#   then the neighborhood will be the same as the borough
#df_combined.loc[df_combined.Neighborhood == 'Not assigned', 'Neighborhood'] = df_combined[df_combined.Neighborhood == 'Not assigned']['Borough']
df_clean = df_combined[df_combined.Neighborhood == 'Not assigned'].reset_index(drop=True)
df_combined.loc[df_clean.index, 'Neighborhood'] = df_combined.loc[df_clean.index, 'Borough']
df_combined.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,North York
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [12]:
# reset index

df_combined.reset_index(drop=True, inplace=True)
df_combined

,PostalCode,Borough,Neighborhood
0,M3A,North York,North York
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Not assigned
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### Adding Coordinates 

In [13]:
df_coor = pd.read_csv("Geospatial_Coordinates.csv")
df_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_full = df_combined.copy()

#df_full.head()
df_full['Latitude'] = np.zeros(len(df_full))
df_full['Longitude'] = np.zeros(len(df_full))
df_full.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,North York,0.0,0.0
1,M4A,North York,Victoria Village,0.0,0.0
2,M5A,Downtown Toronto,Harbourfront,0.0,0.0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",0.0,0.0
4,M7A,Downtown Toronto,Queen's Park,0.0,0.0


In [15]:
for index, row in df_full.iterrows():
    lat = df_coor[df_coor['Postal Code']==row['PostalCode']]['Latitude'].data
    lon = df_coor[df_coor['Postal Code']==row['PostalCode']]['Longitude'].data
    df_full.loc[index, 'Latitude'] = lat
    df_full.loc[index, 'Longitude'] = lon

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Series.data is deprecated and will be removed in a future version
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Series.data is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
df_full

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,North York,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Not assigned,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


### Cluster the Neighborhoods

In [17]:
# check Borough names
df_group = df_full.groupby('Borough').count()
df_group

,PostalCode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,11,11,11,11
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


In [18]:
# work with only boroughs that contain the word Toronto
df_tor = df_full[df_full.Borough.str.endswith('Toronto')].reset_index(drop=True)
df_tor

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259


In [19]:
# define foursqure info
CLIENT_ID = 'DTM4ERJ5PLBBUI13LZJO44E0Z2XV1MMXNSZ4WFXDI2S0CXWR' # your Foursquare ID
CLIENT_SECRET = 'ONQOLBNCXJKP3OF51AWOD1RZC4GYFQRGOFMGZIVEHJKE4X5P' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
venues_tor = getNearbyVenues(names=df_tor['Neighborhood'],
                             latitudes=df_tor['Latitude'],
                             longitudes=df_tor['Longitude']
                             )

Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [25]:
venues_tor.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [26]:
# check how many venues were returned for each neighborhood
venues_tor.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",41,41,41,41,41,41
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",85,85,85,85,85,85
Christie,18,18,18,18,18,18


In [30]:
# one hot encoding
onehot_tor = pd.get_dummies(venues_tor[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot_tor['Neighborhood'] = venues_tor['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot_tor.columns[-1]] + list(onehot_tor.columns[:-1])
onehot_tor = onehot_tor[fixed_columns]

onehot_tor.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# group by neighborhood and calculate the mean of frequency
grouped_tor = onehot_tor.groupby('Neighborhood').mean().reset_index()
grouped_tor.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,...,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.01
1,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00


In [32]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# build the model

# set number of clusters
k = 5

grouped_tor_cluster = grouped_tor.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(grouped_tor_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [33]:
# add clustering labels
grouped_tor.insert(0, 'Cluster Labels', kmeans.labels_)

df_tor_merged = df_tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_tor_merged = df_tor_merged.join(grouped_tor.set_index('Neighborhood'), on='Neighborhood')

df_tor_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,0.021739,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.000000,0.0
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0,0.026316,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.026316,0.0
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,0.000000,0.0,0.0,0.0,...,0.0,0.01,0.0,0.0,0.00,0.01,0.01,0.00,0.000000,0.0
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,0.000000,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.01,0.00,0.00,0.01,0.000000,0.0
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,0.000000,0.0,0.0,0.0,...,0.0,0.00,0.2,0.0,0.00,0.00,0.00,0.00,0.000000,0.0


In [36]:
import folium # map rendering library
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

latitude = df_tor_merged.loc[0, 'Latitude']
longitude = df_tor_merged.loc[0, 'Longitude']

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_tor_merged['Latitude'], df_tor_merged['Longitude'], df_tor_merged['Neighborhood'], df_tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Note**: if the rendered folium map is not shown, please see this [snapshort](https://github.com/briandong/coursera_capstone/blob/master/toronto_cluster.png)